Should be signal in signal out.

In [1]:
# Helper functions
import numpy as np
import tensorflow as tf
from keras import layers
import keras as keras

# GENERATE DILATED LAYER FROM 1D SIGNAL
def signal_to_dilated(signal, dilation, n_channels):
    shape = tf.shape(signal)
    pad_elements = dilation - 1 - (shape[2] + dilation - 1) % dilation
    dilated = tf.pad(signal, [[0, 0], [0, 0], [0, pad_elements], [0, 0]])
    dilated = tf.reshape(dilated, [shape[0],-1,dilation,n_channels])
    return tf.transpose(dilated, perm=[0,2,1,3]), pad_elements


# COLLAPSE DILATED LAYER TO 1D SIGNAL
def dilated_to_signal(dilated, pad_elements, n_channels):
    shape = tf.shape(dilated)
    signal = tf.transpose(dilated, perm=[0,2,1,3])
    signal = tf.reshape(signal, [shape[0],1,-1,n_channels])
    return signal[:,:,:shape[1]*shape[2]-pad_elements,:]


# IDENTITY INITIALIZATION OF CONV LAYERS
def identity_initializer():
    def _initializer(shape, dtype=tf.float32, partition_info=None):
        array = np.zeros(shape, dtype=float)
        cx, cy = shape[0]//2, shape[1]//2
        for i in range(np.minimum(shape[2],shape[3])):
            array[cx, cy, i, i] = 1
        return tf.constant(array, dtype=dtype)
    return _initializer

# “In our experiments, simple training losses (e.g., L1) led to noticeably degraded output quality at lower signal-to-noise ratios (SNRs).” ([Germain et al., 2018, p. 2](zotero://select/library/items/A6D78SNY)) ([pdf](zotero://open-pdf/library/items/P4HPP4P3?page=2&annotation=DZN467TR))
def loss_function(target, current, type):
    if type == 'L2':
        return tf.reduce_mean(tf.square(target-current))

“receptive field of the pipeline is 2^14 + 1 samples, i.e., about 1 s of audio for fs = 16 kHz.” ([Germain et al., 2018, p. 2](zotero://select/library/items/A6D78SNY)) ([pdf](zotero://open-pdf/library/items/P4HPP4P3?page=2&annotation=WTGLQ8JQ))

In [5]:
# De-noising Network
import numpy as np
import tensorflow as tf
from keras import layers
# class WSConv2D(tf.keras.layers.Conv2D):
#     def __init__(self, *args, **kwargs):
#         # Glorot normal initialization is also called Xavier initialization
#         super(WSConv2D, self).__init__(kernel_initializer=tf.keras.initializers.glorot_normal, *args, **kwargs)
#     def standardize_weight(self, kernel_weight, eps):
#         mean = tf.math.reduce_mean(kernel_weight, axis=(0, 1, 2), keepdims=True)
#         #  This function first calculates the mean and variance of the kernel, 
#         var = tf.math.reduce_variance(kernel_weight, axis=(0, 1, 2), keepdims=True)
#         # Then uses these values to compute a scale factor. The scale factor is the reciprocal of the square root of the variance multiplied by the fan-in of the kernel (the number of input connections to each neuron in the layer), and is used to normalize the kernel.
#         fan_in = np.prod(kernel_weight.shape[:-1])
#         gain = self.add_weight(
#             name="gain",
#             shape=(kernel_weight.shape[-1],),
#             initializer="ones",
#             trainable=True,
#             dtype=self.dtype,
#         )
#         scale = (
#             tf.math.rsqrt(
#                 tf.math.maximum(var * fan_in, tf.convert_to_tensor(eps, dtype=self.dtype))
#             )
#             * gain
#         )
#         # Finally, the function subtracts the mean from the kernel and multiplies it by the scale factor to produce the standardized kernel.
#         return kernel_weight * scale - (mean * scale)

#     def call(self, inputs, eps=1e-4):
#         self.kernel.assign(self.standardize_weight(self.kernel, eps))
#         return super().call(inputs)
    
class BatchChannelNormalization(tf.keras.layers.Layer):
    def __init__(self, groups=32, **kwargs):
        self.groups = groups
    # input shape: (batch, height, width, channels)
    def call(self, x):
        original_shape = tf.keras.backend.int_shape(x)
        batch_size = original_shape[0]
        channel_size = original_shape[-1]
        assert channel_size % self.groups == 0 and  channel_size > self.groups\
            , f"channel size {self.channel_size} should be divisible by groups {self.groups} and {channel_size} > {self.groups}"
        group_size  = channel_size // self.groups
        target_shape = [batch_size, -1, self.groups, group_size]
        tf.debugging.assert_equal(tf.reduce_prod((batch_size, original_shape[1], original_shape[2],self.groups, group_size)), tf.reduce_prod(original_shape))
        # $\dot{X}_{...c}=\gamma_{c}^{b}\frac{{X}_{...c}-\hat{\mu}_{c}}{\hat{\sigma}_{c}}+\beta_{c}^{b}$
        x = tf.keras.layers.BatchNormalization(axis=-1)(x)
        # Reshape $\dot{X} to$ \dot{X}\in R^{B\times H \times W \times G \times C/G}$
        x_grouped = tf.keras.layers.Reshape(target_shape=target_shape)(x)
        # \dot{Y}_{b..g.}=\check{\gamma}_{g}^{c}\frac{\dot{X}_{b..g.}-\mu_{b..g.}{\sigma_{b..g.}}+\beta_{g}^{c}
        y = tf.keras.layers.BatchNormalization(axis=-2)(x_grouped)
        # Reshape $\dot{Y}$ to $\dot{Y}\in R^{B\times H \times W \times C}$
        y_ungrouped = tf.keras.layers.Reshape(target_shape=original_shape)(y)
        return y        
class AdaptiveNormalization(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(AdaptiveNormalization, self).__init__(**kwargs)
        self.alpha = tf.Variable(1.0, name='alpha')
        self.beta = tf.Variable(0.0, name='beta')
        self.batch_norm = tf.keras.layers.BatchNormalization(**kwargs)
    def call(self, x):
        return self.alpha * x + self.beta * self.batch_norm(x)
# “point-wise nonlinear leaky rectified linear unit (LReLU) [28] max(0.2x, x)” 
# ([Germain et al., 2018, p. 2]
def LeakyReLU(x):
    return tf.maximum(0.2*x,x)
# Weight Standardization Function
# For use use with tf.keras.layers.Conv2D
# WS + Batch Channel Normalization has been shown to be more effective than Batch Normalization alone
# And allows for lower batch sizes (useful for small datasets w/ large images)
def ws_reg(kernel):
  kernel_mean = tf.math.reduce_mean(kernel, keepdims=True, name='kernel_mean')
  kernel = kernel - kernel_mean
  kernel_std = tf.keras.backend.std(kernel, keepdims=True)
  kernel = kernel / (kernel_std + 1e-5)
  return kernel

n_layers=7 # num of internal layers (13 in paper)
n_channels=32 # number of feature maps (64 in paper)
norm_type="AN"

if norm_type == "AN": # Adaptive Norm
    norm_fn = AdaptiveNormalization
elif norm_type == "SBN": # Std Batch Norm
    norm_fn = layers.BatchNormalization
elif norm_type == "BCN": # Batch Channel Norm
    norm_fn = BatchChannelNormalization(groups=8)
else: # NO LAYER NORMALIZATION
    norm_fn = None
model_input = tf.keras.layers.Input(shape=(None, 1), dtype=tf.float32) # input is a single channel waveform (time, 1)
# N x 1 x 1 
input = tf.expand_dims(model_input, axis=-1) # add a conv feature dimension (batch, time, 1, features)
# 1 x N x 1 
input = tf.transpose(input, [0, 2, 1, 3]) # transpose to (batch, 1, time, features)
for current_layer in range(n_layers):
    if current_layer == 0:
        net = tf.keras.layers.Conv2D(n_channels, kernel_size=[1, 3], activation=LeakyReLU,name='se_conv_%d' % current_layer,padding='SAME')(input)
        # net = tf.keras.layers.Dropout(0.3)(net) # I added this dropout layer
        net = norm_fn(name='se_norm_%d' % current_layer)(net)
    else:
        # The content of each intermediate layer is computed from the previous layer via a dilated convolution with 3 × 1 convolutional kernels
        # “Here, we increase the dilation factor exponentially with depth from 2^0 for the 1st intermediate layer to 2^12 for the 13th one.” ([Germain et al., 2018, p. 2])
        dilation_factor = 2 ** current_layer
        net, pad_elements = signal_to_dilated(net, n_channels=n_channels, dilation=dilation_factor)
        net = tf.keras.layers.Conv2D(n_channels, kernel_size=[1, 3], activation=LeakyReLU,name='se_conv_%d' % current_layer,padding='SAME')(net)
        # net = tf.keras.layers.Dropout(0.1)(net) # I added this dropout layer
        net = norm_fn(name='se_norm_%d' % current_layer)(net)
        net = dilated_to_signal(net, n_channels=n_channels, pad_elements=pad_elements)
net = tf.keras.layers.Conv2D(n_channels, kernel_size=[1, 3], activation=LeakyReLU, name='se_conv_last', padding='SAME')(net)
net = norm_fn(name='se_norm_last')(net)
net = tf.keras.layers.Conv2D(1, kernel_size=[1, 1], activation='sigmoid',
                        name='se_fc_last', padding='SAME')(net)
# undo the transpose and squeeze the feature dimension
output = tf.squeeze(tf.transpose(net, [0, 2, 1, 3]), axis=-1)
model = keras.Model(inputs=model_input, outputs=output)
# model.summary()

In [3]:
# Denoising Network Data Loader and Test/Train Split
import numpy as np
import os
os.environ['XLA_FLAGS'] = '--xla=false'
import tensorflow as tf
data_path = os.path.join(os.getcwd(),'data')
X = np.load(os.path.join(data_path, 'inputs_100_1000_signal.npy'), allow_pickle=True)
X = tf.ragged.stack([tf.constant(x) for x in X], axis=0)
X = tf.expand_dims(X, axis=-1)

Y = np.load(os.path.join(data_path, 'targets_100_1000_signal.npy'), allow_pickle=True)
Y = tf.ragged.stack([tf.constant(y) for y in Y], axis=0)
Y = tf.expand_dims(Y, axis=-1)
dataset = tf.data.Dataset.from_tensor_slices((X, Y))
dataset = dataset.shuffle(seed=70, buffer_size=1000)
data_size = dataset.cardinality().numpy()
train_size = int(0.75 * data_size)

train_set = dataset.take(train_size)
val_set = dataset.skip(train_size)

In [6]:
## Training the Denoising Network
# A larger batch size can lead to faster training, but can also result in less accurate models. A smaller batch size can lead to slower training, but can also result in more accurate models.
# start at 64/128
# In general, you should start with a small number of epochs (e.g. 10-20) and increase the number of epochs until the model begins to overfit the training data. 
tf.debugging.set_log_device_placement(True)
print(tf.config.list_physical_devices('GPU'))
print(tf.test.is_built_with_cuda())
print(tf.test.gpu_device_name())

from keras import callbacks
import matplotlib.pyplot as plt
import os
SE_LOSS_LAYERS = 6 # NUMBER OF FEATURE LOSS LAYERS
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=10000, 
    decay_rate=0.95,
)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
def L1_loss(y_true, y_pred):
    return tf.reduce_mean(tf.abs(y_pred - y_true)) 
def L2_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_pred - y_true)) 
custom_objects = tf.keras.utils.get_custom_objects()
custom_objects['L1'] = L1_loss
custom_objects['L2'] = L2_loss
LOSS = 'L1'
model.compile(loss="L1", optimizer=optimizer, metrics=['mse', 'mae', 'accuracy' ])
model_path = os.path.join(os.getcwd(),'saved','models', f'{LOSS}_L{n_layers}C{n_channels}FCNN_{norm_type}.model')
if os.path.exists(model_path):
    model.load_weights(model_path)
    print('Model loaded from: ', model_path)
checkpoint = callbacks.ModelCheckpoint(model_path, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True,mode='min')
stop = callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
# callbacks=[checkpoint, stop]
history = model.fit(train_set, epochs=25, validation_data=val_set, batch_size=32,callbacks=[checkpoint, stop],verbose=1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
True
/device:GPU:0
Epoch 1/25
 97/750 [==>...........................] - ETA: 11:43 - loss: 0.1397 - mse: 0.0534 - mae: 0.1397 - accuracy: 5.1496e-06

In [ ]:
# model_path = os.path.join(os.getcwd(),'saved','models', 'L1_FCNN_AN.model')
# model.save(model_path)

In [15]:
import librosa
import numpy as np
import IPython.display as ipd
idx = 100
demo_tensor = X[idx]
demo_target = Y[idx]
model_input = tf.expand_dims(demo_tensor, axis=0)
prediction = model.predict(model_input).squeeze()
input = demo_tensor.numpy().squeeze()
target = demo_target.numpy().squeeze()
import soundfile as sf
sf.write('input.wav', input, 16000)
sf.write('prediction.wav', prediction, 16000)
sf.write('prediction.wav', target, 16000)
print(L1_loss(target, prediction).numpy())
ipd.Audio(prediction, rate=16000)

1/1 [==============================] - 0s 38ms/step
0.046649706


“To compute the loss between two waveforms, we apply a pretrained audio classification network to each waveform and compare the internal activation patterns induced in the network by the two signals” ([Germain et al., 2018, p. 1](zotero://select/library/items/A6D78SNY)) ([pdf](zotero://open-pdf/library/items/P4HPP4P3?page=1&annotation=Y3F49L4C))

“The network consists of 15 convolutional layers with 3×1 kernels, batch normalization, LReLU units, and zero padding” ([Germain et al., 2018, p. 2](zotero://select/library/items/A6D78SNY)) ([pdf](zotero://open-pdf/library/items/P4HPP4P3?page=2&annotation=J3JNI54Q))


In [ ]:
# FEATURE LOSS NETWORK
import keras as keras
# TODO
# Still in the proccess of converting to keras
# still need to train the network
# still need to figure out how to connect the feature loss network to the main network
# “The number of channels is doubled every 5 layers, with 32 channels in the first intermediate layer.” 
n_layers=14
base_channels=32
doubling_rate=5
conv_layers = []
# input 4D tensor
model_input = tf.keras.layers.Input(shape=(None, 1, None, None), dtype=tf.float32) 
# STRCUTURE OF THE FEATURE LOSS NETWORK VERY SIMILAR TO THE MAIN NETWORK
for current_layer in range(n_layers):
    n_channels = base_channels * (2 ** (current_layer // doubling_rate)) # UPDATE CHANNEL COUNT
    if current_layer == 0:
        # "Each Layer is decimated by 2" - just means stride of 2 in the time dimension.
        net = layers.Conv2D( n_channels, kernel_size=[1, 3], activation=LeakyReLU, stride=[1, 2],
                            name='loss_conv_%d'%current_layer, padding='SAME')(model_input)
        net = layers.BatchNormalization(net)
        conv_layers.append(net)
    elif current_layer < n_layers - 1:
        net = layers.Conv2D(conv_layers[-1], n_channels, kernel_size=[1, 3], activation=LeakyReLU,
                            stride=[1, 2], name='loss_conv_%d'%current_layer, padding='SAME')
        net = layers.BatchNormalization(net)
        conv_layers.append(net)
    else:
        net = layers.Conv2D(conv_layers[-1], n_channels, kernel_size=[1, 3], activation=LeakyReLU,
                            name='loss_conv_%d'%current_layer, padding='SAME')
        net = layers.BatchNormalization(net)
        conv_layers.append(net)
        # TODO
        # "Each channel in the last layer is averaged-pooled to produce the output ferature vector."
# TODO
# The logistic classifier, which is a component of the network that is used to make predictions about the audio data, is trained specifically for each individual task. This allows the network to learn task-specific information and improve its performance on each task.
    # Then need to train the model on 2 seperate tasks
    # inner CNN layers (n_layers) w/ different output layers
    # Assuming this will be either a softmax or sigmoid layer depending on the task
#EG
# output1 = keras.layers.Dense(...)(x)
# output1 = keras.layers.Activation('softmax', name='output1')(output1)
# 
# output2 = keras.layers.Dense(...)(x)
# output2 = keras.layers.Activation('softmax', name='output2')(output2)

# # Compile the model at this point.
# model.compile(
#     optimizer=...,
#     loss={'output1': ..., 'output2': ...},
#     metrics={'output1': ..., 'output2': ...},
# )

# Train the model on the data for each task
model.fit(x=x1, y=y1,)
model.fit(x=x2, y=y2, task=2)
def FeatureLoss(target, current, loss_weights, loss_layers):

    feat_current = LossNetwork(current,reuse=False, n_layers=n_layers, norm_type=norm_type)
    
    feat_target = LossNetwork(target, reuse=True, n_layers=n_layers, norm_type=norm_type)
    
    #“The weights λm are set to balance the contribution of each layer to the loss. They are set to the inverse of the relative values of ‖Φm(ß) − Φm(g(x; θ))‖1 after 10 training epochs. (For these first 10 epochs, the weights are set to 1.)”
    loss_vec = []
    for id in range(loss_layers):
        loss_vec.append(loss_function(feat_current[id], feat_target[id], type="L1") / loss_weights[id])
    # b) Denoising loss function:
    for id in range(1,loss_layers+1):
        loss_vec[0] += loss_vec[id]
    
    return loss_vec
